In [1]:
import pandas as pd
import numpy as np
import os
import sys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

In [3]:
from global_functions import remove_spaces_in_columns, up_load_df

In [4]:
#בתי ספר ממשרד החינוך בכל הארץ
Gschool=up_load_df(r'{}\background_files\ministry_of_education\מוסדות'.format(education_folder_path),'schools_2023')
Gschool = remove_spaces_in_columns(Gschool)

In [5]:
#יישובים בשטח צתאל שם וקוד יישוב
JTMT_setls=up_load_df(r'{}\background_files'.format(education_folder_path),'210615_מקוצר_רשימת_יישובים_באחריות_צתאל')
JTMT_setls = remove_spaces_in_columns(JTMT_setls)

In [6]:
Gschool=Gschool[Gschool['יישוב'].isin(JTMT_setls['שם_יישוב'])]

In [7]:
Gschool=Gschool[Gschool['יישוב'] != 'ירושלים']
Gschool=Gschool[Gschool['יישוב'] != 'בית שמש']

In [8]:
#קאורדינטות של מוסדות חינוך בכל הארץ ממשרד החינוך
moe_mosdot_coordinates_2022=up_load_df(r'{}\background_files\ministry_of_education\מוסדות'.format(education_folder_path),'moe_mosdot_coordinates_2022')
moe_mosdot_coordinates_2022=remove_spaces_in_columns(moe_mosdot_coordinates_2022)

In [9]:
moe_mosdot_coordinates_2022.rename(columns={'SEMEL_MOSAD': 'סמל_מוסד'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'SHEM_MOSAD': 'שם מוסד'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'ITM_X': 'coordinate_x'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'ITM_Y': 'coordinate_y'}, inplace=True)

moe_mosdot_coordinates_2022 = moe_mosdot_coordinates_2022.drop(columns=['UTM_X', 'UTM_Y', 'RAMAT_DIYUK_MIKUM'])

In [10]:
# מיזוג הטבלאות לפי העמודה "סמל_מוסד"
Gschool = pd.merge(Gschool, moe_mosdot_coordinates_2022[['סמל_מוסד', 'coordinate_x', 'coordinate_y']],
                     on='סמל_מוסד', how='left')

In [11]:
Gschool.head()

,סמל_מוסד,שם_מוסד,יישוב,כתובת,"כתובת_חט""ב",טלפון,"דוא""ל_מזכירות",סוג_חינוך,מעמד_משפטי,סוג_פיקוח,מגזר,יחידת_דיווח,שכבה,מספר_תלמידים,coordinate_x,coordinate_y
0,110098,השחר,טל שחר,טל שחר,NaN,08-9349345,NaN,רגיל,רשמי,ממלכתי,יהודי,ירושלים,א - ו,420.00,190867.00,634787.00
1,110726,מתתיהו,צלפון,צלפון 5,NaN,02-9911647,sigalyeh1@hinuchm.k12.il,רגיל,רשמי,ממלכתי דתי,יהודי,ירושלים,א - ו,349.00,194187.00,634711.00
2,110734,הר טוב,צרעה,צרעה 28,NaN,02-9908900,NaN,רגיל,רשמי,ממלכתי,יהודי,ירושלים,א - ו,513.00,197528.00,630138.00
3,111096,עין הרים,"עין כרם-בי""ס חקלאי","עין כרם-בי""ס חקלא 5",NaN,02-6431318,yaelelk1@hinuchm.k12.il,רגיל,רשמי,ממלכתי,יהודי,ירושלים,א - ו,545.00,214636.00,631692.00
4,111195,חברון בנות,קרית ארבע,נחל אשכול 407,NaN,02-9961050,zipporahapo@hinuchm.k12.il,רגיל,רשמי,ממלכתי דתי,יהודי,ירושלים,א - ו,293.00,211257.00,604302.00


In [12]:
Gschool=Gschool.drop(columns=['כתובת', 'כתובת_חט"ב', "טלפון", 'דוא"ל_מזכירות', "מעמד_משפטי", "יחידת_דיווח"])

In [14]:
# שמירת רק השורות שבהן הערך בעמודה "כיתה_א" הוא NaN (מחיקת שורות שבהן יש ערך)
df = Gschool[Gschool['coordinate_x'].isna()]
df

,סמל_מוסד,שם_מוסד,יישוב,סוג_חינוך,סוג_פיקוח,מגזר,שכבה,מספר_תלמידים,coordinate_x,coordinate_y
64,115600,ארץ הצבר,מודיעין-מכבים-רעות,רגיל,ממלכתי,יהודי,א - ו,154.00,NaN,NaN
66,115675,צלילים,מודיעין-מכבים-רעות,מיוחד,ממלכתי,יהודי,גן - יב,9.00,NaN,NaN
133,160077,דרכי שלום,גבעת יערים,רגיל,חרדי,יהודי,ט - יב,NaN,NaN,NaN
378,484204,שש שנתי תורני לבנים נופים,נופים,רגיל,ממלכתי דתי,יהודי,ז - יב,0.00,NaN,NaN
437,632323,תורת חיים ביתר,ביתר עילית,רגיל,חרדי,יהודי,א - ד,80.00,NaN,NaN
441,637231,"בית ספר חב""ד יצהר",יצהר,רגיל,חרדי,יהודי,א - ו,NaN,NaN,NaN
455,655233,שירת ציון,כוכב יעקב,רגיל,חרדי,יהודי,ט - י,28.00,NaN,NaN
460,658880,ישיבת היכל התורה,ביתר עילית,רגיל,חרדי,יהודי,ט - יא,NaN,NaN,NaN
461,658898,ישיבה תיכונית צביה,משואה,רגיל,ממלכתי דתי,יהודי,ז - יב,87.00,NaN,NaN
462,658948,ארץ מולדת,מודיעין-מכבים-רעות,רגיל,ממלכתי,יהודי,א - ו,219.00,NaN,NaN


In [13]:
Gschool=Gschool[Gschool['coordinate_x'] != 0]

In [ ]:
# שמירת רק השורות שבהן המילה "גן" מופיעה בעמודה "שם_מוסד"
df = Gschool[Gschool['שכבה'].str.contains('גן', na=False)]
df

In [15]:
Gschool=Gschool[~Gschool['שכבה'].str.contains('גן', na=False)]

In [ ]:
Gschool.head()

In [ ]:
is_unique = Gschool['סמל_מוסד'].is_unique
is_unique